In [1]:
import numpy as np 
from matplotlib import pyplot as plt

/home/mahmoud/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
from colorama import Style, Fore, Back
blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
grn_bck = Back.GREEN
res = Style.RESET_ALL

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Exercise 8-1.
> The inverse of the inverse is the original matrix; in other words, A−1 analgous to how 1/ 1/a = a. Illustrate this using Python.

In [5]:
A = np.array([[1, 2], [3, 4]])
A_inv = np.linalg.inv(A)
A_inv_inv = np.linalg.inv(A_inv)

print(f"{blk}A{res}:\n{A}")
print(f"{blk}A_inv_inv{res}:\n{A_inv_inv}")

A:
[[1 2]
 [3 4]]
A_inv_inv:
[[1. 2.]
 [3. 4.]]


# Exercise 8-2.
> Implement the full algorithm described in “Inverting Any Square Full-Rank Matrix” on page 134 and reproduce Figure 8-3. Of course, your matrices will look different from Figure 8-3 because of random numbers, although the grid and identity matrices will be the same.

In [17]:
def get_inverse (mat : np.ndarray, plot : bool = True) -> np.ndarray:
    
    # minors matrix 
    minors = np.zeros_like(mat)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            minors[i,j] = np.linalg.det(mat[np.arange(mat.shape[0])!=i][:,np.arange(mat.shape[1])!=j])

    # cofactor matrix
    cofactor = np.zeros_like(mat)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            cofactor[i,j] = (-1)**(i+j) * minors[i,j]

    # adjugate matrix
    adjugate = cofactor.T

    # determinant
    det = np.linalg.det(mat)

    # inverse matrix
    inv = adjugate / det

    if plot:
        fig = make_subplots(rows=2, cols=3, start_cell="top-left")

        fig.add_trace(go.Heatmap
            (z = mat, colorscale='gray', showscale=False),
            row=1, col=1
        )
        fig.add_trace(go.Heatmap
            (z = minors, colorscale='gray', showscale=False),
            row=1, col=2
        )
        fig.add_trace(go.Heatmap
            (z = cofactor, colorscale='gray', showscale=False),
            row=1, col=3
        )
        fig.add_trace(go.Heatmap
            (z = adjugate, colorscale='gray', showscale=False),
            row=2, col=1
        )
        fig.add_trace(go.Heatmap
            (z = inv, colorscale='gray', showscale=False),
            row=2, col=2
        )

        fig.update_layout(height=600, width=800, title_text="Inverse Matrix Calculation")
        fig.show()

    return inv


A = np.random.randint(0, 10, (10, 10))
A_inv_ = get_inverse(A)
A_inv_inv = get_inverse(A_inv_)


# check if the inverse of the inverse almost equals the original matrix

if np.absolute(A_inv_inv - A).max() < 1e-6:
    print(f"{grn_bck}The inverse of the inverse almost equals the original matrix{res}")
else:

    print(f"{red}The inverse of the inverse does not equal the original matrix{res}")


The inverse of the inverse almost equals the original matrix


# Exercise 8-3.
> Implement the full-inverse algorithm by hand for a 2 × 2 matrix using matrix elements a, b, c, and d. I don’t normally assign hand-solved problems in this book, but this exercise will show you where the shortcut comes from. Remember that the determinant of a scalar is its absolute value.

In [18]:
A = np.array([[1, 2], [3, 4]])
a, b, c, d = A.flatten()

A_inv = np.array([[d, -b], [-c, a]]) / (a*d - b*c)
a, b, c, d = A_inv.flatten()

A_inv_inv = np.array([[d, -b], [-c, a]]) / (a*d - b*c)

print(f"{blk}A{res}:\n{A}")
print(f"{blk}A_inv_inv{res}:\n{A_inv_inv}")

A:
[[1 2]
 [3 4]]
A_inv_inv:
[[1. 2.]
 [3. 4.]]


# Exercise 8-4.
> Derive the right-inverse for wide matrices by following the logic that allowed us to discover the left-inverse. Then reproduce Figure 8-4 for a wide matrix. (Hint: start from the code for the left-inverse and adjust as necessary.)

In [32]:
wide_matrix = np.random.rand(3, 15)

w_inv = wide_matrix.T @ np.linalg.inv(wide_matrix@wide_matrix.T)


I = wide_matrix @ w_inv

print(I.round(14))

[[ 1.  0. -0.]
 [-0.  1.  0.]
 [-0.  0.  1.]]


# Exercise 8-5.
> Illustrate in Python that the pseudoinverse (via np.linalg.pinv) equals the full inverse (via np.linalg.inv) for an invertible matrix. Next, illustrate that the pseudoinverse equals the left-inverse for a tall full column-rank matrix, and that it equals the right-inverse for a wide full row-rank matrix.

In [43]:
# full rank 
A = np.random.rand(3, 3)
A_inv = np.linalg.inv(A)
A_pinv = np.linalg.pinv(A)
print((A_inv-A_pinv).max()<1e-5)

T = np.random.rand(15, 3)
L = np.linalg.inv(T.T @ T) @ T.T
L_pinv = np.linalg.pinv(T)
print((L-L_pinv).max()<1e-5)


W = np.random.rand(3, 15)
R = W.T @ np.linalg.inv(W @ W.T)
R_pinv = np.linalg.pinv(W)
print((R-R_pinv).max()<1e-5)


True
True
True


# Exercise 8-6.
> The LIVE EVIL rule applies to the inverse of multiplied matrices. Test this in code by creating two square full-rank matrices A and B, then use Euclidean distance to compare (1) AB , (2) A−1B−1, and (3) B−1A−1. Before starting to code, make a prediction about which results will be equal. Print out your results using formatting like the following (I’ve omitted my results so you won’t be biased!): Distance between (AB)^-1 and (A^-1)(B^-1) is ___ Distance between (AB)^-1 and (B^-1)(A^-1) is ___ As an extra challenge, you can confirm that the LIVE EVIL rule applies to a longer string of matrices, e.g., four matrices instead of two.

In [45]:
A = np.random.rand(4, 4)
B = np.random.rand(4, 4)
A_inv = np.linalg.inv(A)
B_inv = np.linalg.inv(B)

AB_inv = np.linalg.inv(A @ B)
A_invB_inb = A_inv @ B_inv
B_invA_inb = B_inv @ A_inv


print("Is (AB)^-1 = A^-1 * B^-1: ", (AB_inv-A_invB_inb).max()<1e-5)
print("Is (AB)^-1 = B^-1 * A^-1: ", (AB_inv-B_invA_inb).max()<1e-5)

Is (AB)^-1 = A^-1 * B^-1:  False
Is (AB)^-1 = B^-1 * A^-1:  True
